In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rasterio
!pip install geetools
import rasterio as rio
import numpy as np
from PIL import Image
import rasterio
import copy
import os
import datetime
import ee, geetools, time
from google.cloud import storage

In [ ]:

centralDataset=[]
# date=datetime.datetime.today()
c_year = 2024
date=datetime.date(c_year, 7, 1)
sec="https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FFLDAS%2FFLDAS_NOAHMP001_G_CA_D.001%2F2023%2F01%2FFLDAS_NOAHMP001_G_CA_D.A20230101.001.nc&SHORTNAME=FLDAS_NOAHMP001_G_CA_D&VARIABLES=Evap_tavg&SERVICE=L34RS_LDAS&FORMAT=dGlmLw&DATASET_VERSION=001&LABEL=FLDAS_NOAHMP001_G_CA_D.A20230101.001.nc.SUB.tif&BBOX=24.562%2C86.152%2C25.153%2C86.612&VERSION=1.02%27"
ET = []
centralnames=[]

USERNAME_GESDISC = "xxxxx" # TODO: Replace with original credentials
PASSWORD_GESDISC= "xxxxx" # TODO: Replace with original credentials
path = f'/content/drive/MyDrive/ET_downloads/{c_year}_{c_year+1}'
  if not os.path.exists(path):
      os.makedirs(path)

for i in range(365):
    year=str(date.year)
    month=str(date.month).zfill(2)
    day=str(date.day).zfill(2)
    fullDate = date.strftime('%Y%m%d')
    filename = "https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FFLDAS%2FFLDAS_NOAHMP001_G_CA_D.001%2F"+year+"%2F"+month+"%2FFLDAS_NOAHMP001_G_CA_D.A"+fullDate+".001.nc&SERVICE=L34RS_LDAS&BBOX=21%2C65.566%2C37.932%2C99.844&FORMAT=Y29nLw&VERSION=1.02&SHORTNAME=FLDAS_NOAHMP001_G_CA_D&LABEL=FLDAS_NOAHMP001_G_CA_D.A"+fullDate+".001.nc.SUB.tif&DATASET_VERSION=001&VARIABLES=Evap_tavg"
    print(filename)
    outputName = f"{path}/"+fullDate+".tif"
    print(date.strftime('%Y%m%d'), year, month, day, filename)
    ! wget -O $outputName --user USERNAME_GESDISC --password PASSWORD_GESDISC "$filename"
    date += datetime.timedelta(days=1)
    centralnames.append(outputName)


https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FFLDAS%2FFLDAS_NOAHMP001_G_CA_D.001%2F2024%2F07%2FFLDAS_NOAHMP001_G_CA_D.A20240701.001.nc&SERVICE=L34RS_LDAS&BBOX=21%2C65.566%2C37.932%2C99.844&FORMAT=Y29nLw&VERSION=1.02&SHORTNAME=FLDAS_NOAHMP001_G_CA_D&LABEL=FLDAS_NOAHMP001_G_CA_D.A20240701.001.nc.SUB.tif&DATASET_VERSION=001&VARIABLES=Evap_tavg
https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FFLDAS%2FFLDAS_NOAHMP001_G_CA_D.001%2F2024%2F07%2FFLDAS_NOAHMP001_G_CA_D.A20240701.001.nc&SERVICE=L34RS_LDAS&BBOX=21%2C65.566%2C37.932%2C99.844&FORMAT=Y29nLw&VERSION=1.02&SHORTNAME=FLDAS_NOAHMP001_G_CA_D&LABEL=FLDAS_NOAHMP001_G_CA_D.A20240701.001.nc.SUB.tif&DATASET_VERSION=001&VARIABLES=Evap_tavg
https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FFLDAS%2FFLDAS_NOAHMP001_G_CA_D.001%2F2024%2F07%2FFLDAS_NOAHMP001_G_CA_D.A20240701.001.nc&SERVICE=L34RS_LDAS&BBOX=21%2C65.566%2C37.932%2C99.

In [ ]:
len(centralnames)

365

# The following cell contains code for aggregating all the ET files for a year into 1 annual tif

In [ ]:
res=0

index=0
with rasterio.open(centralnames[0]) as src1:
  image1 = src1.read()
  profile1 = src1.profile

for i in range(1, 365):
    try:
      with rasterio.open(centralnames[i]) as src2:
        image2 = src2.read()
        profile2 = src2.profile
    except:
      continue
    print(centralnames[i])
    if image1.shape != image2.shape:
      raise ValueError("Images must have the same shape")
    # Add the images together
    result = image1 + image2
    image1 = copy.deepcopy(result)
    # Update the profile with the result's shape
profile1.update(count=result.shape[0])


# Write the resulting image with the preserved CRS
with rasterio.open(f'/content/drive/MyDrive/ET_downloads/{c_year}_{c_year+1}/result_{c_year}_{c_year+1}_Hydroyear.tif', 'w', **profile1) as dst:
    dst.write(result)


with rasterio.open(f'/content/drive/MyDrive/ET_downloads/{c_year}_{c_year+1}/{c_year}0701.tif') as src:
    with rasterio.open(f'/content/drive/MyDrive/ET_downloads/{c_year}_{c_year+1}/result_{c_year}_{c_year+1}_Hydroyear.tif', 'r+', **src.profile) as dst:
        dst.write(dst.read())

/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.tif
/content/drive/MyDrive/ET_downloads/2024_2025/20240701.t

# The following cell contains code for aggregating all the ET files for a year into 26 fortnights

In [ ]:
res=0

index=0
for s in range(0, 363, 14):
  for i in range(s+1, s+14):
    # print(i, end="->")
      if(i==s+1):
        try:
          with rasterio.open(f'/content/drive/MyDrive/ET_downloads/{c_year}_{c_year+1}/{centralnames[i-1]}') as src1:
            image1 = src1.read()
            profile1 = src1.profile
        except:
          # with rasterio.open('/content/'+centralnames[i-1]) as src1:
          #   image1 = src1.read()
          #   profile1 = src1.profile
          # i+=1
          continue
      try:
        with rasterio.open(f'/content/drive/MyDrive/ET_downloads/{c_year}_{c_year+1}/{centralnames[i]}') as src2:
          image2 = src2.read()
          profile2 = src2.profile
      except:
        # i+=1
        continue
      print(centralnames[i])
      # Add the images together
      result = image1 + image2
      image1 = copy.deepcopy(result)
      # Update the profile with the result's shape
  profile1.update(count=result.shape[0])

  # Read the first image


  # Read the second image

  # Check if the images have the same shape
  if image1.shape != image2.shape:
      raise ValueError("Images must have the same shape")

  path = f'/content/drive/MyDrive/ET_downloads/{c_year}_{c_year+1}/fortnight'
  if not os.path.exists(path):
      os.makedirs(path)

  # Write the resulting image with the preserved CRS
  with rasterio.open(f'{path}/Hydro_{c_year}_{c_year+1}_{str(index)}.tif', 'w', **profile1) as dst:
      dst.write(result)

  # Copy the CRS value from the source image to the destination image
  with rasterio.open(f'/content/drive/MyDrive/ET_downloads/{c_year}_{c_year+1}/{c_year+1}0114.tif') as src:
      with rasterio.open(f'{path}/Hydro_{c_year}_{c_year+1}_{str(index)}.tif', 'r+', **src.profile) as dst:
          dst.write(dst.read())
  index+=1

  # print(index+1,"==================================================")
  # index+=1
# # for i in range(1, len(centralnames)):

# Code to export tif to GEE

In [ ]:
ee.Authenticate()
ee.Initialize(project='corestack-datasets')

client = storage.Client()
bucket = client.get_bucket('core_stack')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.1 MB/s eta 0:00:00


In [ ]:
#Drive to GEE
def upload_file_to_gcs(local_file_path, file_name):
    blob = bucket.blob(f'ET_FLDAS/{file_name}.tif') # GCS path
    blob.upload_from_filename(local_file_path)
    print("Upload complete.")


def upload_tif_from_gcs_to_gee(file_name):
  # CSV GCS path
  gcs_path = f'gs://core_stack/ET_FLDAS/{file_name}.tif'

  # Create task ID and run table ingestion
  task_id = ee.data.newTaskId()[0]
  asset_id = f'projects/corestack-datasets/assets/datasets/ET_FLDAS/{file_name}'

  manifest = {
        'name': asset_id,
        'tilesets': [{
            'sources': [{
                'uris': [gcs_path]
            }]
        }],
        'bands': [{
            'id': 'b1'
        }]
    }

  task = ee.data.startIngestion(task_id, manifest)
  print("Ingestion task started:", task_id, task)
  check_task_status([task['id']])
  make_asset_public(asset_id)

Make GEE asset public

In [ ]:
from google.api_core import retry
def make_asset_public(asset_id):
    try:
        # Get the ACL of the asset
        acl = ee.data.getAssetAcl(asset_id)

        # Add 'all_users' to readers
        acl["all_users_can_read"] = True

        # Update the ACL
        @retry.Retry()
        def update_acl():
            ee.data.setAssetAcl(asset_id, acl)

        update_acl()

        # Verify the change
        updated_acl = ee.data.getAssetAcl(asset_id)
        if updated_acl.get("all_users_can_read"):
            print(f"Successfully made asset {asset_id} public")
            return True
        else:
            print(f"Failed to make asset {asset_id} public")
            return False
    except Exception as e:
        print(f"Error making asset public: {str(e)}")
        return False

Check GEE task status and exit when completed

In [ ]:
def check_task_status(task_id_list, sleep_time=60):
    task_id_list = list(filter(None, task_id_list))
    if len(task_id_list) > 0:
        time.sleep(sleep_time)
        tasks = ee.data.listOperations()
        if tasks:
            for task in tasks:
                task_id = task["name"].split("/")[-1]
                if task_id in task_id_list and task["metadata"]["state"] in (
                    "SUCCEEDED",
                    "COMPLETED",
                    "FAILED",
                    "CANCELLED",
                ):
                    task_id_list.remove(task_id)
        print("task_id_list after", task_id_list)

        if len(task_id_list) > 0:
            print("Tasks not completed yet...")
            check_task_status(task_id_list)
    return task_id_list

Export annual data to GEE

In [ ]:
file_path = f"/content/drive/MyDrive/ET_downloads/{c_year}_{c_year+1}/result_{c_year}_{c_year+1}_Hydroyear.tif"
file_name = f"ET_Hydroyear/ET_{c_year}_{c_year+1}_Hydroyear"

upload_file_to_gcs(file_path, file_name)
upload_tif_from_gcs_to_gee(file_name)

Ingestion task started: dc845526-bc6f-4f8e-8f39-49fc8844b19b {'id': 'VBQWNANPRCA7MVAASGSB4WOZ', 'name': 'projects/corestack-datasets/operations/VBQWNANPRCA7MVAASGSB4WOZ', 'started': 'OK'}
task_id_list after []
Successfully made asset projects/corestack-datasets/assets/datasets/ET_FLDAS/ET_Hydroyear/ET_2024_2025_Hydroyear public


Export fortnightly data to GEE

In [ ]:
from glob import glob
dist_data_path = f'/content/drive/MyDrive/ET_downloads/{c_year}_{c_year+1}/fortnight/'

files = glob(dist_data_path + "*.tif")
print("no. of files:", len(files), '\n')

for file_path in files:
  print(file_path)
  file_name = file_path.split("/")[-1].split(".")[0]
  file_name = f"ET_fortnight/{file_name}"
  print(file_name)
  upload_file_to_gcs(file_path, file_name)
  upload_tif_from_gcs_to_gee(file_name)

no. of files: 26 

/content/drive/MyDrive/ET_downloads/2024_2025/fortnight/Hydro_2024_2025_0.tif
ET_fortnight/Hydro_2024_2025_0
Upload complete.
Ingestion task started: c84d32aa-d5c2-4254-a847-0e4835411fb9 {'id': '5DKUEGNDXJ3OQIHNOOBFJMC5', 'name': 'projects/corestack-datasets/operations/5DKUEGNDXJ3OQIHNOOBFJMC5', 'started': 'OK'}
task_id_list after ['5DKUEGNDXJ3OQIHNOOBFJMC5']
Tasks not completed yet...
task_id_list after []
Successfully made asset projects/corestack-datasets/assets/datasets/ET_FLDAS/ET_fortnight/Hydro_2024_2025_0 public
/content/drive/MyDrive/ET_downloads/2024_2025/fortnight/Hydro_2024_2025_1.tif
ET_fortnight/Hydro_2024_2025_1
Upload complete.
Ingestion task started: 0c5a4c46-42b9-493b-9b4a-7f3700dbfb71 {'id': 'KSKDMMAQKN32JUGXETXONZVZ', 'name': 'projects/corestack-datasets/operations/KSKDMMAQKN32JUGXETXONZVZ', 'started': 'OK'}
task_id_list after []
Successfully made asset projects/corestack-datasets/assets/datasets/ET_FLDAS/ET_fortnight/Hydro_2024_2025_1 public
/con

Following is the code for aggregation of 1 fortnight (**No need to run this**)

In [ ]:
# res=0

# index=0
# # for s in range(0, min(363, len(centralnames)), 14):
# for i in range(0, 14):
#   # print(i, end="->")

#     if(i==0):
#       with rasterio.open('/content/'+centralnames[i-1]) as src1:
#         image1 = src1.read()
#         profile1 = src1.profile

#     with rasterio.open('/content/'+centralnames[i]) as src2:
#       image2 = src2.read()
#       profile2 = src2.profile
#     print(centralnames[i])
#     # Add the images together
#     result = image1 + image2
#     image1 = copy.deepcopy(result)
#     # Update the profile with the result's shape
# profile1.update(count=result.shape[0])

# # Read the first image


# # Read the second image

# # Check if the images have the same shape
# if image1.shape != image2.shape:
#     raise ValueError("Images must have the same shape")



# # Write the resulting image with the preserved CRS
# with rasterio.open('result_'+str(index)+'.tif', 'w', **profile1) as dst:
#     dst.write(result)

# # Copy the CRS value from the source image to the destination image
# with rasterio.open('/content/'+str(centralnames[-1])) as src:
#     with rasterio.open('result_'+str(index)+'.tif', 'r+', **src.profile) as dst:
#         dst.write(dst.read())
# index+=1

# print("==================================================")
# # for i in range(1, len(centralnames)):
